In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio as rio

import argparse
import json
import os

from contextlib import contextmanager
from functools import partial
from rasterio import crs, transform, warp, enums, features
from africa_tools import log_print, rasterize, geotiff_writer, extract_geo_info, reproject, ID

CURVE = 'curve'
USE_SECTOR = 'USE_SECTOR'
VALUE_COLUMN = 'VALFIS'

use_sector_table = {
        "emp_serv":  "SERV",
        "emp_agr":  "AGR",
        "emp_gov":  "GOV",
        "emp_ind":  "IND",
        "ic_low":    "RES_LI", 
        "ic_high":    "RES_MHI", 
        "ic_mhigh":    "RES_MHI", 
        "ic_mlow":    "RES_MHI"
}
band_order = ["RES_LI", "RES_MHI", "SERV", "AGR", "GOV", "IND"]
curve_name_mapping = {
    "I_T1(m).fvu": "T1",
    "I_M1(m).fvu": "M1",
    "I_M2(m).fvu": "M2",
    "I_W1(m).fvu": "W1",
    "I_C1(m).fvu": "C1"
}

def override_config(config_file):
    config_obj = json.load(config_file)
    use_sector_table = config_obj['use_sector_table']
    band_order = config_obj['band_order']
    curve_name_mapping = config_obj['curve_name_mapping']

def init_parser():       
    parser = argparse.ArgumentParser(description='''
        Process a shapefile and extracts the % of use by curve.
        Writes a tiff file for each curve''',
        epilog='''
        Usage example:       
        python extract_use.py shapefile.shp config.json mask_file.tiff output_dir''')
    
    parser.add_argument('shapefile', help='shapefile to process',
        type=argparse.FileType('r'))
    
    parser.add_argument('config',
                        help='json file with the "use" configuration',
                        type=argparse.FileType('r'))

    parser.add_argument('mask',
                        help='geotiff file for regrid',
                        type=argparse.FileType('r'))
    
    parser.add_argument('outdir', 
                        help='output directory',
                        type=str)
    return parser

def get_use_on_df(df, idx):
    r = df.iloc[idx]
    if r[USE_SECTOR] in use_sector_table:
        return use_sector_table[r[USE_SECTOR]] 
    else:
        return r[USE_SECTOR]

def check_dataframe(df):
    all_columns = True
    for C in (VALUE_COLUMN, ID, USE_SECTOR, CURVE):
        if C not in df:
            print(f'ERROR: column {C} not in shapefile')
            all_columns = False
    if not all_columns:
        sys.exit(-1)
        
    unique_use = df[USE_SECTOR].unique()
    missing_use = filter(
        lambda u: u not in use_sector_table, 
        unique_use
    )
    if len(list(missing_use))>0:
        log_print(f'warning missing uses: [{", ".join(missing_use)}]')

In [68]:
shapefile = 'E:/africa_downscaling/sz/SZ_buiA_GAR_5km_onlyBU_20180604/SZ_buiA_GAR_5km_onlyBU_20180604.shp'

log_print('loading file')
df = gpd.read_file(shapefile)

check_dataframe(df)

log_print('getting geometries')    

geometries = df                                         \
                .groupby(ID)                            \
                .agg({'geometry':'first'})



log_print('aggregate on use')            
get_use = partial(get_use_on_df, df)
df_abs_values = df                                          \
                .loc[                                       \
                    df[USE_SECTOR].isin(use_sector_table)   \
                ]                                           \
                .groupby([CURVE, get_use, ID])              \
                .agg({VALUE_COLUMN: 'sum'})

log_print('calculating percentage')
def fix_percentage(x):
    x_perc = round(100 * x/x.sum())
    if (x_perc.sum()[0])<100:
        err = 100-x_perc.sum()        
        x_perc.loc[x_perc.idxmax()] += err
    return x_perc

df_perc_values = df_abs_values                              \
                 .groupby(level=(CURVE,ID))                 \
                 .apply(fix_percentage)


df_perc_check = df_perc_values                              \
                 .groupby(level=(CURVE,ID))                 \
                 .apply(lambda x: 100 == x.sum())  

            

2018-08-03 16:16:03 - loading file
2018-08-03 16:16:03 - getting geometries
2018-08-03 16:16:03 - aggregate on use
2018-08-03 16:16:04 - calculating percentage


In [69]:
df_perc_check

VALFIS
curve       ID_5X5           
I_C1(m).fvu 8523482.0    True
            8531116.0    True
            8531117.0    True
            8531118.0    True
            8533023.0    True
            8533024.0    True
            8559734.0    True
I_M1(m).fvu 8523482.0    True
            8531116.0    True
            8531117.0    True
            8531118.0    True
            8533023.0    True
            8533024.0    True
            8559734.0    True
I_M2(m).fvu 8498698.0    True
            8502516.0    True
            8502523.0    True
            8506319.0    True
            8506320.0    True
            8506327.0    True
            8506328.0    True
            8506329.0    True
            8508231.0    True
            8508233.0    True
            8508236.0    True
            8510135.0    True
            8510136.0    True
            8510137.0    True
            8510141.0    True
            8513944.0    True
...                       ...
I_T1(m).fvu 8563577.0    True
            8565466.0    True
            8565467.0    True
            8565468.0    True
            8565469.0    True
            8565470.0    True
            8565471.0    True
            8565472.0    True
            8565473.0    True
            8565474.0    True
            8565475.0    True
            8565476.0    True
            8565477.0    True
            8565478.0    True
            8565479.0    True
            8565480.0    True
            8565481.0    True
            8567369.0    True
            8567370.0    True
            8567371.0    True
            8567372.0    True
            8567373.0    True
            8567374.0    True
            8567375.0    True
            8567376.0    True
            8567377.0    True
            8567378.0    True
            8567379.0    True
            8567380.0    True
            8567381.0    True

[904 rows x 1 columns]

In [42]:
x.argsort()

array([1, 2, 0], dtype=int64)

In [65]:
x_perc = df_perc_values.loc[('I_C1(m).fvu', slice(None), 8523482.0)]
x_perc.idxmax()

VALFIS    (I_C1(m).fvu, RES_MHI, 8523482.0)
dtype: object

In [ ]:
x_perc